In [ ]:
# Install MySQL Connector for Python
# pip install mysql-connector-python
# pip install sqlalchemy
# pip install PyMySQL

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
# MySQL connection details
# function to connect to MySQL database
def create_mysql_engine(user, password, host, database):
    connection_string = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return create_engine(connection_string)    

engine = create_mysql_engine('root', 'password', 'localhost', 'retailoltp')

In [5]:
# Read tables into DataFrames
customers_df = pd.read_sql("SELECT * FROM Customers", engine)
products_df = pd.read_sql("SELECT * FROM Products", engine)
orders_df = pd.read_sql("SELECT * FROM Orders", engine)
orderdetails_df = pd.read_sql("SELECT * FROM OrderDetails", engine)

In [6]:
customers_df.head()

,CustomerID,FirstName,LastName,Email,Phone,Region
0,1,David,Valenzuela,juancollins@gmail.com,418-278-7191,South\r
1,2,Thomas,Lane,starkstephanie@pearson.net,001-035-708-6633x45,South\r
2,3,Lisa,Leonard,taylorjasmine@gmail.com,001-249-255-3670,West\r
3,4,Chad,Pierce,rosariomegan@gmail.com,144-233-0233x75679,West\r
4,5,Gregory,Fields,christina44@pacheco.com,(778)293-2081x3578,South\r


## EDA (Exploratory Data Analysis)

Check shape and preview data:

In [7]:
print(customers_df.shape)
print(products_df.describe())   # Summary statistics
print(orders_df.info())         # Data types

(50, 6)
        ProductID    UnitPrice    CostPrice
count  100.000000   100.000000   100.000000
mean    50.500000   920.314800   626.337900
std     29.011492   547.062796   373.889951
min      1.000000    18.100000     9.070000
25%     25.750000   492.150000   341.645000
50%     50.500000   864.640000   568.525000
75%     75.250000  1356.930000   896.322500
max    100.000000  1954.680000  1604.720000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   OrderID     5000 non-null   int64 
 1   CustomerID  5000 non-null   int64 
 2   OrderDate   5000 non-null   object
 3   Status      5000 non-null   object
dtypes: int64(2), object(2)
memory usage: 156.4+ KB
None


Check for missing values:

In [8]:
print("customers_df missing values:")
print(customers_df.isnull().sum())

print("\n\nproducts_df missing values:")
print(products_df.isnull().sum())

customers_df missing values:
CustomerID    0
FirstName     0
LastName      0
Email         0
Phone         0
Region        0
dtype: int64


products_df missing values:
ProductID      0
ProductName    0
Category       0
UnitPrice      0
CostPrice      0
dtype: int64


### Data Cleaning

In [9]:
print(customers_df.head())
customers_df.info()

   CustomerID FirstName     LastName                        Email  \
0           1     David   Valenzuela        juancollins@gmail.com   
1           2    Thomas         Lane   starkstephanie@pearson.net   
2           3      Lisa      Leonard      taylorjasmine@gmail.com   
3           4      Chad       Pierce       rosariomegan@gmail.com   
4           5   Gregory       Fields      christina44@pacheco.com   

                  Phone    Region  
0          418-278-7191   South\r  
1   001-035-708-6633x45   South\r  
2      001-249-255-3670    West\r  
3    144-233-0233x75679    West\r  
4    (778)293-2081x3578   South\r  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  50 non-null     int64 
 1   FirstName   50 non-null     object
 2   LastName    50 non-null     object
 3   Email       50 non-null     object
 4   Phone       50 non-nul

In [10]:
# customer region column clean up
customers_df['Region'] = customers_df['Region'].str.replace('\r', '').str.strip()

customers_df.head()

,CustomerID,FirstName,LastName,Email,Phone,Region
0,1,David,Valenzuela,juancollins@gmail.com,418-278-7191,South
1,2,Thomas,Lane,starkstephanie@pearson.net,001-035-708-6633x45,South
2,3,Lisa,Leonard,taylorjasmine@gmail.com,001-249-255-3670,West
3,4,Chad,Pierce,rosariomegan@gmail.com,144-233-0233x75679,West
4,5,Gregory,Fields,christina44@pacheco.com,(778)293-2081x3578,South


In [11]:
# Strip extra quotes/spaces
products_df["ProductName"] = products_df["ProductName"].str.replace("'", "").str.strip()
products_df["Category"] = products_df["Category"].str.replace("'", "").str.strip()

products_df.head()

,ProductID,ProductName,Category,UnitPrice,CostPrice
0,1,Data Sports,Furniture,171.66,149.30
1,2,Improve Books,Electronics,810.97,580.66
2,3,West Books,Fashion,884.96,788.11
3,4,Store Fashion,Fashion,663.15,454.51
4,5,Medical Appliances,Appliances,1346.10,825.69


In [12]:
# Fill missing values
products_df["CostPrice"] = products_df["CostPrice"].fillna(products_df["CostPrice"].mean())

products_df.head()

,ProductID,ProductName,Category,UnitPrice,CostPrice
0,1,Data Sports,Furniture,171.66,149.30
1,2,Improve Books,Electronics,810.97,580.66
2,3,West Books,Fashion,884.96,788.11
3,4,Store Fashion,Fashion,663.15,454.51
4,5,Medical Appliances,Appliances,1346.10,825.69


In [15]:
print(orders_df.head())
orders_df.info()

   OrderID  CustomerID   OrderDate        Status
0        1          16  2025-06-03     Pending\r
1        2          16  2025-05-12   Cancelled\r
2        3          30  2024-10-05   Completed\r
3        4          46  2024-11-19     Pending\r
4        5          47  2024-10-08   Cancelled\r
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   OrderID     5000 non-null   int64 
 1   CustomerID  5000 non-null   int64 
 2   OrderDate   5000 non-null   object
 3   Status      5000 non-null   object
dtypes: int64(2), object(2)
memory usage: 156.4+ KB


In [16]:
# Ensure data types
orders_df["OrderDate"] = pd.to_datetime(orders_df["OrderDate"])

# oders status column clean up
orders_df['Status'] = orders_df['Status'].str.replace('\r', '').str.strip()

orders_df.info()
orders_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OrderID     5000 non-null   int64         
 1   CustomerID  5000 non-null   int64         
 2   OrderDate   5000 non-null   datetime64[ns]
 3   Status      5000 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 156.4+ KB


,OrderID,CustomerID,OrderDate,Status
0,1,16,2025-06-03,Pending
1,2,16,2025-05-12,Cancelled
2,3,30,2024-10-05,Completed
3,4,46,2024-11-19,Pending
4,5,47,2024-10-08,Cancelled


In [17]:
# checking orderdetails_df quantity dtype
orderdetails_df["Quantity"] = orderdetails_df["Quantity"].astype(int)

orderdetails_df.info()
orderdetails_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OrderDetailID  10000 non-null  int64  
 1   OrderID        10000 non-null  int64  
 2   ProductID      10000 non-null  int64  
 3   Quantity       10000 non-null  int64  
 4   Discount       10000 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 390.8 KB


,OrderDetailID,OrderID,ProductID,Quantity,Discount
0,1,440,4,3,17.92
1,2,1809,74,2,24.24
2,3,194,27,5,36.85
3,4,2636,87,4,46.64
4,5,1061,68,5,39.08


### Create Dimension Tables

In [18]:
# DimCustomer

# combine first and last names into full name
customers_df["FullName"] = customers_df["FirstName"] + " " + customers_df["LastName"]

# rename CustomerID to CustomerKey for dimensional modeling
customers_df.rename(columns={"CustomerID": "CustomerKey"}, inplace=True)

# drop columns first name and last name
dim_customer = customers_df.drop(columns=["FirstName", "LastName"])

dim_customer.head()

,CustomerKey,Email,Phone,Region,FullName
0,1,juancollins@gmail.com,418-278-7191,South,David Valenzuela
1,2,starkstephanie@pearson.net,001-035-708-6633x45,South,Thomas Lane
2,3,taylorjasmine@gmail.com,001-249-255-3670,West,Lisa Leonard
3,4,rosariomegan@gmail.com,144-233-0233x75679,West,Chad Pierce
4,5,christina44@pacheco.com,(778)293-2081x3578,South,Gregory Fields


In [19]:
# DimProduct
dim_product = products_df.rename(columns={"ProductID": "ProductKey"})

dim_product.head()

,ProductKey,ProductName,Category,UnitPrice,CostPrice
0,1,Data Sports,Furniture,171.66,149.30
1,2,Improve Books,Electronics,810.97,580.66
2,3,West Books,Fashion,884.96,788.11
3,4,Store Fashion,Fashion,663.15,454.51
4,5,Medical Appliances,Appliances,1346.10,825.69


In [20]:
# DimDate (extract from Orders)
dim_date = pd.DataFrame()
dim_date["OrderDate"] = pd.to_datetime(orders_df["OrderDate"])
dim_date["DateKey"] = dim_date["OrderDate"].dt.strftime("%Y%m%d").astype(int)
dim_date["Year"] = dim_date["OrderDate"].dt.year
dim_date["Month"] = dim_date["OrderDate"].dt.month
dim_date["Day"] = dim_date["OrderDate"].dt.day
dim_date.drop_duplicates(inplace=True)

dim_date.head().sort_values(by="DateKey")

,OrderDate,DateKey,Year,Month,Day
2,2024-10-05,20241005,2024,10,5
4,2024-10-08,20241008,2024,10,8
3,2024-11-19,20241119,2024,11,19
1,2025-05-12,20250512,2025,5,12
0,2025-06-03,20250603,2025,6,3


In [21]:
print(dim_date.columns)
print(dim_customer.columns)
print(dim_product.columns)

Index(['OrderDate', 'DateKey', 'Year', 'Month', 'Day'], dtype='object')
Index(['CustomerKey', 'Email', 'Phone', 'Region', 'FullName'], dtype='object')
Index(['ProductKey', 'ProductName', 'Category', 'UnitPrice', 'CostPrice'], dtype='object')


In [22]:
# Create Fact Table

# Join Orders + OrderDetails
order_merge = pd.merge(orderdetails_df, orders_df, on="OrderID", how="inner")

# Add keys
order_merge["DateKey"] = pd.to_datetime(order_merge["OrderDate"]).dt.strftime("%Y%m%d").astype(int)


order_merge.head()

# Select fact columns
# fact_sales = fact_sales[[
#     "OrderDetailID", "CustomerID", "ProductID", "DateKey", 
#     "Quantity", "UnitPrice", "CostPrice", "Status"
# ]]

,OrderDetailID,OrderID,ProductID,Quantity,Discount,CustomerID,OrderDate,Status,DateKey
0,1,440,4,3,17.92,33,2025-07-06,Cancelled,20250706
1,2,1809,74,2,24.24,31,2025-08-04,Pending,20250804
2,3,194,27,5,36.85,2,2025-01-14,Completed,20250114
3,4,2636,87,4,46.64,18,2025-04-06,Completed,20250406
4,5,1061,68,5,39.08,40,2025-05-11,Pending,20250511


In [23]:
# join with customers to get CustomerKey
order_customer = pd.merge(
    order_merge, 
    dim_customer[["CustomerKey"]],
    left_on="CustomerID", 
    right_on="CustomerKey", 
    how="right")

print(order_customer.size)
order_customer.head()

100000


,OrderDetailID,OrderID,ProductID,Quantity,Discount,CustomerID,OrderDate,Status,DateKey,CustomerKey
0,19,4604,72,2,27.59,1,2025-06-01,Completed,20250601,1
1,74,2080,15,3,10.75,1,2024-11-22,Cancelled,20241122,1
2,103,3475,4,2,39.97,1,2025-03-30,Pending,20250330,1
3,115,4929,57,4,9.29,1,2025-02-21,Pending,20250221,1
4,151,3236,36,1,44.39,1,2025-08-04,Pending,20250804,1


In [24]:
# drop redundant CustomerID
order_customer.drop(columns=["CustomerID"], inplace=True)

In [25]:
# join with order_customers to get ProductKey
fact_sales_tmp = pd.merge(
    order_customer, 
    dim_product[["ProductKey", "UnitPrice", "CostPrice"]],
    left_on="ProductID", 
    right_on="ProductKey", 
    how="right")

print(fact_sales_tmp.size)
fact_sales_tmp.head()

120000


,OrderDetailID,OrderID,ProductID,Quantity,Discount,OrderDate,Status,DateKey,CustomerKey,ProductKey,UnitPrice,CostPrice
0,2171,900,1,1,5.96,2025-05-11,Pending,20250511,1,1,171.66,149.3
1,2533,4962,1,5,7.56,2024-09-26,Completed,20240926,1,1,171.66,149.3
2,5654,2692,1,4,17.13,2025-04-22,Cancelled,20250422,2,1,171.66,149.3
3,2498,4999,1,2,9.99,2025-08-17,Pending,20250817,3,1,171.66,149.3
4,4163,4480,1,5,11.52,2024-12-04,Completed,20241204,4,1,171.66,149.3


In [26]:
# drop productID and orderID
fact_sales_tmp.drop(columns=["ProductID", "OrderID"], inplace=True)


In [27]:
# Metrics
fact_sales_tmp["SalesAmount"] = fact_sales_tmp["Quantity"] * fact_sales_tmp["UnitPrice"]
fact_sales_tmp["TotalCost"] = fact_sales_tmp["Quantity"] * fact_sales_tmp["CostPrice"]
fact_sales_tmp["Profit"] = fact_sales_tmp["SalesAmount"] - fact_sales_tmp["TotalCost"]

# drop duplicates
fact_sales_tmp.drop_duplicates(inplace=True)

# Rename columns for dimensional modeling
fact_sales_tmp.rename(columns={"OrderDetailID": "SalesFactKey"}, inplace=True)

# drop columns not needed in fact table
fact_sales = fact_sales_tmp.drop(columns=["UnitPrice", "CostPrice"])

# arrange columns
fact_sales = fact_sales[[
    "SalesFactKey", "CustomerKey", "ProductKey", "DateKey", 
    "Quantity", "SalesAmount", "TotalCost", "Profit", "Status"
]]

fact_sales.head()
fact_sales.info()
fact_sales.size

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SalesFactKey  10000 non-null  int64  
 1   CustomerKey   10000 non-null  int64  
 2   ProductKey    10000 non-null  int64  
 3   DateKey       10000 non-null  int64  
 4   Quantity      10000 non-null  int64  
 5   SalesAmount   10000 non-null  float64
 6   TotalCost     10000 non-null  float64
 7   Profit        10000 non-null  float64
 8   Status        10000 non-null  object 
dtypes: float64(3), int64(5), object(1)
memory usage: 703.3+ KB


90000

In [28]:
fact_sales.head()

,SalesFactKey,CustomerKey,ProductKey,DateKey,Quantity,SalesAmount,TotalCost,Profit,Status
0,2171,1,1,20250511,1,171.66,149.3,22.36,Pending
1,2533,1,1,20240926,5,858.30,746.5,111.80,Completed
2,5654,2,1,20250422,4,686.64,597.2,89.44,Cancelled
3,2498,3,1,20250817,2,343.32,298.6,44.72,Pending
4,4163,4,1,20241204,5,858.30,746.5,111.80,Completed


### Load Data to DWH

In [ ]:
# connect with reatildwh schema

engine = create_mysql_engine('root', 'password', 'localhost', 'retaildwh')

In [ ]:
# write to MySQL
dim_customer.to_sql('DimCustomer', con=engine, if_exists='replace', index=False)
dim_product.to_sql('DimProduct', con=engine, if_exists='replace', index=False)
dim_date.to_sql('DimDate', con=engine, if_exists='replace', index=False)
fact_sales.to_sql('FactSales', con=engine, if_exists='replace', index=False)